<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Практика" data-toc-modified-id="Практика-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Практика</a></span><ul class="toc-item"><li><span><a href="#Загрузка-DataFrame" data-toc-modified-id="Загрузка-DataFrame-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка DataFrame</a></span><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Задача 1</a></span></li></ul></li><li><span><a href="#Теория-вероятности.-События" data-toc-modified-id="Теория-вероятности.-События-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Теория вероятности. События</a></span><ul class="toc-item"><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Задача 3</a></span></li></ul></li></ul></li></ul></div>

# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')))
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
# Ваше решение

df["Норма"] = np.nan
df["Норма"] = df.Норма.astype(str)
pr_norm = (df[df['Норматив'].str.contains('пределах', case=False)])
pr_norm["nov"] = pr_norm['Норматив'].str.split(expand=True)[2]
pr_norm["nov_1"] = pr_norm['nov'].apply(lambda x: x.split()[0].split('-')[0])
pr_norm["nov_2"] = pr_norm['nov'].apply(lambda x: x.split()[0].split('-')[1])
pr_norm["nov_1"] = pr_norm.nov_1.astype(int)
pr_norm["nov_2"] = pr_norm.nov_2.astype(int)

for i in pr_norm.index:
    if pr_norm.at[i, 'nov_1'] <= float(pr_norm.at[i, 'Результат анализа']) <= pr_norm.at[i, 'nov_2']:
        df.at[i, 'Норма'] = 'Да'
    else:
        df.at[i, 'Норма'] = 'Нет'
        
pr_norm_colour = (df[df['Норматив'].str.contains('более 30', case=False)])
df.at[pr_norm_colour.index, 'Норма'] = 'Да'
pr_norm_1 = (df[df['Норматив'].str.contains('более', case=False)])
pr_norm_1 = pr_norm_1.drop(index=[2])
pr_norm_1["nov"] = pr_norm_1['Норматив'].apply(lambda x: x.split()[-1].split(' ')[0])
pr_norm_1["nov"] = pr_norm_1['Норматив'].apply(lambda x: x.split()[-1].split('-')[-1])
pr_norm_1["nov"] = pr_norm_1["nov"].str.replace(',', '.').astype(float)

for i in pr_norm_1.index:
    if float(pr_norm_1.at[i, 'Результат анализа']) < pr_norm_1.at[i, 'nov']:
        df.at[i, 'Норма'] = 'Да'
    else:
         df.at[i, 'Норма'] = 'Нет'

<ipython-input-3-2c8a11a65a8e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_norm["nov"] = pr_norm['Норматив'].str.split(expand=True)[2]
<ipython-input-3-2c8a11a65a8e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pr_norm["nov_1"] = pr_norm['nov'].apply(lambda x: x.split()[0].split('-')[0])
<ipython-input-3-2c8a11a65a8e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [6]:
df = df.set_index('Показатель')
df

,Единица измерений,Результат анализа,Норматив,Норма
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Да
Запах,баллы,1,не более 2-3,Да
Цветность,градусы,б/цвета,не более 30,Да
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Да
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Да
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Да
Нитраты (по NO3),мг/дм3,24,не более 45,Да
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Да
Хлориды (Cl),мг/дм3,200,не более 350,Да


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [85]:
# Ваше решение

ver_pop_1 = 5/9 # вероятность вытаскивания 1-ого апельсина
ver_pop_2 = 4/8 # вероятность вытаскивания 2-ого апельсина
ver_pop_3 = 3/7 # вероятность вытаскивания 3-ого апельсина
conditions = ['Yes', 'No'] # список возможных значений вытаскивантя, апельсин "Yes", не апельсин "No"
N =10000 # число вытаскиваний

In [86]:
popytka_1 = np.random.choice(conditions, N, p=[ver_pop_1, 1 - ver_pop_1])                          

popytka_1

array(['No', 'Yes', 'Yes', ..., 'Yes', 'No', 'Yes'], dtype='<U3')

In [87]:
popytka_2 = np.random.choice(conditions, N, p=[ver_pop_2, 1 - ver_pop_2])

popytka_2

array(['Yes', 'No', 'No', ..., 'Yes', 'No', 'No'], dtype='<U3')

In [88]:
popytka_3 = np.random.choice(conditions, N, p=[ver_pop_3, 1 - ver_pop_3])

popytka_3

array(['No', 'No', 'Yes', ..., 'No', 'Yes', 'No'], dtype='<U3')

In [89]:
df = pd.DataFrame({'popytka_1': popytka_1,
                   'popytka_2': popytka_2,
                   'popytka_3': popytka_3})
df

,popytka_1,popytka_2,popytka_3
0,No,Yes,No
1,Yes,No,No
2,Yes,No,Yes
3,Yes,Yes,No
4,Yes,No,No
...,...,...,...
9995,Yes,No,No
9996,Yes,No,No
9997,Yes,Yes,No
9998,No,No,Yes


In [90]:
itog = 0
for i in df.index:
        if df.at[i, 'popytka_1'] == df.at[i, 'popytka_2'] == df.at[i, 'popytka_3'] == 'Yes':
            itog = itog + 1
itog

1218

In [94]:
print('Вероятность, что все три фрукта – апельсины: ', itog/N)

Вероятность, что все три фрукта – апельсины:  0.1218


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [5]:
# Ваше решение